In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image 


def plot_disks(points, table_size_cm=(100, 50), r_cm=1, dpi=100, scale_factor=0.1, name:str="SHU_Pattern"):
    """
    Plot a pattern of disks on a table.
    
    Parameters:
    - points: list of (x, y) positions in cm
    - table_size_cm: (width, height) in cm
    - r_cm: radius of each disk in cm
    - dpi: resolution of the figure
    - scale_factor: used to rescale for screen viewing (0.1 means 10% of actual size)
    """
    table_width_cm, table_height_cm = table_size_cm

    # Create figure in inches for 1:1 scaling
    fig_width_in = table_width_cm * scale_factor  
    fig_height_in = table_height_cm * scale_factor

    fig, ax = plt.subplots(figsize=(fig_width_in, fig_height_in), dpi=dpi)
    ax.set_xlim(-table_width_cm/2, table_width_cm/2)
    ax.set_ylim(-table_height_cm/2, table_height_cm/2)
    ax.set_aspect('equal')
    ax.set_facecolor('black')

    for (x, y) in points:
        xs=x
        ys=y
        rs=r_cm
        circle = patches.Circle((xs,ys), radius=rs, edgecolor='white', facecolor='white', alpha=1)
        ax.add_patch(circle)
        ax.plot([xs - rs, xs + rs], [ys, ys], color='black', linewidth=1)  # horizontal cross
        ax.plot([xs, xs], [ys - rs, ys + rs], color='black', linewidth=1)  # vertical cross

    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    rectangle = patches.Rectangle(
    (-table_width_cm / 2 + 5, -table_height_cm / 2 + 5),  # bottom-left
    table_width_cm - 2 * 5,  # width
    table_height_cm - 2 * 5,  # height
    edgecolor='red',
    facecolor='none',
    linewidth=2
    )    
    ax.add_patch(rectangle)


    # Remove axis spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    plt.grid(True, which='both', linestyle='-', linewidth=2.5)
    plt.tight_layout()
    plt.savefig(rf"{name}.png", dpi=dpi, bbox_inches='tight', pad_inches=0)

    plt.close()




In [2]:
from natsort import natsorted
from pathlib import Path
import os
import numpy as np

sample = 1

folder_path=rf"H:\phd stuff\tidy3d\structures\2D SHU"
count = 0
for l,file in enumerate(natsorted(os.listdir(folder_path))):
    print(Path(file).stem)
    if not Path(file).stem in ["chi_0.37_N_10000_posics"]:
        continue
    data = []
    centers = []

    with open(os.path.join(folder_path, file),"r") as f: 
        lines = f.read().splitlines() 
        i=0
        index = 1
        num_samples = int(lines[index].split()[0])

        while num_samples>0 and i < sample+1:
            try:
                num_samples = int(lines[index].split()[0])
                info = lines[index]
                old_index = index
                index += num_samples+1
                li = (lines[old_index+1:index])
                centers = np.array([list(map(float, item.split())) for item in li])
                data += [{
                    "centers":centers,
                    "Lx":float(lines[old_index].split()[1]),
                    "Ly":float(lines[old_index].split()[2]),
                    "points":float(lines[old_index].split()[0]),
                }]
                i+=1
            except:
                num_samples = 0
    

chi_0.20_N_10000_posics
chi_0.30_N_10000_posics
chi_0.31_N_10000_posics
chi_0.32_N_10000_posics
chi_0.33_N_10000_posics
chi_0.34_N_10000_posics
chi_0.35_N_10000_posics
chi_0.36_N_10000_posics
chi_0.37_N_10000_posics
chi_0.38_N_10000_posics
chi_0.39_N_10000_posics
chi_0.40_N_10000_posics


In [3]:
slicing = 0.07
centers = data[0]["centers"].T
a=1.587
slab_size_x = 55*a
radius=0.189*a

centers_scaled = centers * a
x, y = centers_scaled

# Apply the conditions
mask = (np.abs(y) <= (np.max(y) * slicing)) & (np.abs(x) <= (slab_size_x / 2))

# Filtered coordinates
filtered_centers = np.copy(centers_scaled)[:, mask]
# filtered_centers[1,:]-=8*a
filtered_centers[0,:]+=5

In [4]:
np.shape(filtered_centers) #x,y positions of the cylinders

(2, 382)

In [5]:
plot_disks(filtered_centers.T, scale_factor=0.23*0.95,r_cm=radius, table_size_cm=(114.5,57.5),dpi=100)

In [6]:
Original_Image = Image.open("./SHU_Pattern.png") 
rotated_image = Original_Image.rotate(0.65, expand=False, resample=Image.BICUBIC)
rotated_image.save("SHU_Pattern_2.png")

In [7]:
# import cv2
# import numpy as np

# # Load distorted image
# image = cv2.imread("SHU_Pattern.png")

# # 1. Manually or programmatically get the distorted corners (e.g., from a calibration pattern)
# # Example: corners in the image (in pixel coordinates)
# pts_src = np.array([
#     [100, 15],   # top-left
#     [2253, 15],   # top-right
#     [2253, 1092],  # bottom-right
#     [100, 1092]   # bottom-left
# ], dtype=np.float32)
# pixels_cm_y=21.54
# pixels_cm_x=10.77
# # 2. Define the corresponding points on the corrected image
# pts_dst = np.array([
#     [100-pixels_cm_x*0.6, 15],   # top-left
#     [2253, 15],   # top-right
#     [2253-0.3*pixels_cm_x*0.3, 1092],  # bottom-right
#     [100, 1092-pixels_cm_y]   # bottom-left
# ], dtype=np.float32)

# # 3. Compute the homography matrix
# h, _ = cv2.findHomography(pts_src, pts_dst)

# # 4. Warp the image using the homography
# corrected_image = cv2.warpPerspective(image, h, (2300, 1150))

# # 5. Save or show the result
# cv2.imwrite("corrected_grid.jpg", corrected_image)



In [8]:
#54.3 x 108


In [9]:
108/107.3

1.0065237651444547

In [10]:
pixels_y = 1092-15 #50cm
pixels_cm_y = pixels_y/50
pixels_cm_y

21.54

In [11]:
pixels_x = 2253-100 #100cm
pixels_cm_x = pixels_y/100
pixels_cm_x

10.77

In [12]:
np.max(y) * slicing

5.555899300749001